In [ ]:
import os

def create_river_folders(base_directory):
    """
    Create folders for river monitoring stations in the specified directory.
    
    Parameters:
    base_directory (str): The path where folders should be created
    """

    # Create the base directory if it doesn't exist
    if not os.path.exists(base_directory):
        os.makedirs(base_directory)
        print(f"Created base directory: {base_directory}")
    
    # Create each folder
    created_count = 0
    skipped_count = 0
    
    for folder_name in folder_names:
        folder_path = os.path.join(base_directory, folder_name)
        
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            created_count += 1
            print(f"Created: {folder_name}")
        else:
            skipped_count += 1
            print(f"Already exists: {folder_name}")
    
    print(f"\n✓ Summary: {created_count} folders created, {skipped_count} already existed")
    print(f"Total folders: {len(folder_names)}")

# Run it
target_directory = r"E:\Dissertation\Data\RiverMapping\Reaches"
create_river_folders(target_directory)

In [ ]:
import os
import geopandas as gpd
from pathlib import Path

def convert_gpkg_to_shapefile():
    """
    Convert .gpkg files to shapefiles with specific attributes and reprojection.
    Searches through all planform types and river names, converts each .gpkg to a shapefile
    with a single 'ds_order' attribute set to 1, reprojects to EPSG:3395, and saves to
    the corresponding reach folder.
    """
    
    # Define paths
    source_base = r"E:\Dissertation\Data\Zhao_etal_2025"
    target_base = r"E:\Dissertation\Data\RiverMapping\Reaches"
    
    # Define planform types
    planform_types = ["B", "HSW", "LSW", "Me"]
    
    # Counters for summary
    success_count = 0
    error_count = 0
    not_found_count = 0
    errors = []
    
    # Get list of all river folders from the target directory
    river_folders = [f for f in os.listdir(target_base) 
                    if os.path.isdir(os.path.join(target_base, f))]
    
    print(f"Found {len(river_folders)} river reach folders")
    print(f"Searching through {len(planform_types)} planform types\n")
    
    # Process each river and planform combination
    for river_name in river_folders:
        found_gpkg = False
        
        for planform in planform_types:
            # Construct source path
            source_dir = os.path.join(source_base, planform, river_name)
            source_file = os.path.join(source_dir, f"{river_name}.gpkg")
            
            # Check if the .gpkg file exists
            if os.path.exists(source_file):
                found_gpkg = True
                
                try:
                    # Read the geopackage
                    gdf = gpd.read_file(source_file)
                    
                    # Verify it has exactly one feature
                    if len(gdf) != 1:
                        print(f"⚠ Warning: {river_name} ({planform}) has {len(gdf)} features, expected 1")
                    
                    # Create new GeoDataFrame with only ds_order attribute
                    gdf_new = gpd.GeoDataFrame(
                        {'ds_order': [1]},
                        geometry=gdf.geometry,
                        crs=gdf.crs
                    )
                    
                    # Reproject to EPSG:3395 (WGS 84 / World Mercator)
                    gdf_reprojected = gdf_new.to_crs('EPSG:3395')
                    
                    # Construct output path
                    output_dir = os.path.join(target_base, river_name)
                    output_file = os.path.join(output_dir, f"{river_name}.shp")
                    
                    # Save as shapefile
                    gdf_reprojected.to_file(output_file)
                    
                    success_count += 1
                    print(f"✓ Converted: {river_name} (from {planform})")
                    
                except Exception as e:
                    error_count += 1
                    error_msg = f"{river_name} ({planform})"
                    errors.append(error_msg)
                    print(f"✗ Error: {river_name} ({planform}) - {str(e)}")
                    print(f"   Skipping and continuing with remaining rivers...")
                
                # Break after finding first matching planform (whether success or error)
                break
        
        if not found_gpkg:
            not_found_count += 1
            print(f"○ Not found: {river_name} (checked all planform types)")
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    print(f"✓ Successfully converted: {success_count}")
    print(f"○ GPKG files not found: {not_found_count}")
    print(f"✗ Errors encountered: {error_count}")
    print(f"Total river folders processed: {len(river_folders)}")
    
    if errors:
        print(f"\n{'='*60}")
        print(f"FAILED RIVERS")
        print(f"{'='*60}")
        for error in errors:
            print(f"  - {error}")

if __name__ == "__main__":
    print("Starting GPKG to Shapefile conversion...\n")
    convert_gpkg_to_shapefile()
    print("\nConversion process complete!")

In [ ]:
import os

target_base = r"E:\Dissertation\Data\RiverMapping\Reaches"

print(f"Does directory exist? {os.path.exists(target_base)}")
if os.path.exists(target_base):
    folders = [f for f in os.listdir(target_base) if os.path.isdir(os.path.join(target_base, f))]
    print(f"Number of folders found: {len(folders)}")
    if folders:
        print(f"First few folders: {folders[:5]}")

In [ ]:
import os

def get_folder_names_from_directory(directory_path):
    """
    Generate a list of folder names from a given directory.
    
    Parameters:
    directory_path (str): Path to the directory containing folders
    
    Returns:
    list: Sorted list of folder names
    """
    # Get all items in the directory
    all_items = os.listdir(directory_path)
    
    # Filter to only keep directories (not files)
    folder_names = [item for item in all_items 
                   if os.path.isdir(os.path.join(directory_path, item))]
    
    # Sort alphabetically for consistency
    folder_names.sort()
    
    return folder_names

# Use it
directory = r"E:\Dissertation\Data\Zhao_etal_2025\B"
folder_names = get_folder_names_from_directory(directory)

# Print as a Python list that you can copy-paste
print(f"Found {len(folder_names)} folders:\n")
print("folder_names = [")
for i, name in enumerate(folder_names):
    if i < len(folder_names) - 1:
        print(f"    '{name}',")
    else:
        print(f"    '{name}'")
print("]")

# Also print just the count and first few examples
print(f"\nTotal: {len(folder_names)} folders")
if folder_names:
    print(f"First 5: {folder_names[:5]}")
    print(f"Last 5: {folder_names[-5:]}")

## This folder creates folders with river names at the given directory
It will optionally generate an additional subfolder, such as "cleaned" below the river_name folder

In [ ]:
import os

def create_unique_river_folders(base_directory, subfolder_name=None):
    """
    Create folders for river monitoring stations, avoiding duplicates.
    
    Parameters:
    base_directory (str): The path where folders should be created
    subfolder_name (str, optional): If provided, creates a subfolder with this name
                                    inside each river folder
    """
    
    # Raw list of folder names with duplicates
    raw_folder_names = [
        'Amazonas_Tamshiyacu', 'AmuDarya_Kerki', 'Amur_Khabarovsk',
        'Amyl_Kachulka', 'Apalachicola_NearBlountstown', 'Araguaia_Aruana',
        'Araguaia_LuizAlves', 'Araguaia_SaoFelixDoAraguaia', 'Beas_MandiPlain',
        'Beni_Rurrenabaque', 'Benue_Ibi', 'Bhareli_NTRoadCrossing',
        'Brahmaputra_Bahadurabad', 'Brahmaputra_Pandu', 'Brahmaputra_Yangcun',
        'Chenab_Akhnoor', 'Cuiaba_PortoDoAlegre', 'Demini_PostoAjuricaba',
        'Gandak_Dumariaghat', 'Gandak_Triveni', 'Ganges_Paksey',
        'Guapore_Pimenteiras', 'HuangHe_Huayuankou', 'HuangHe_TanglaiQu',
        'Ica_IpirangaVelho', 'Indus_Kotri', 'Irrawaddy_Katha',
        'Irrawaddy_Sagaing', 'Itacuai_LadarioJusante', 'Jurua_EirunepeMontante',
        'Jurua_SantosDumont', 'Katun_Srostki', 'Kokcha_Khojaghar',
        'Krishna_Vijayawada', 'Lena_Tabaga', 'Liard_UpperCrossing',
        'Logone_Bongor', 'Logone_Lai', 'Madeira_Humaita',
        'Mamore_Guajara-Mirim', 'Mamore_PuertoSiles', 'Maranon_Borja',
        'Maranon_SanRegis', 'Mortes_SantoAntonioDoLeverger', 'Napo_Bellavista',
        'Napo_NvoRocafuerte', 'Naryn_UstKekirim', 'Ob_Prokhorkino',
        'Panj_NizPyandzh', 'Parana_Corrientes', 'Peace_FifthMeridian',
        'Pilcomayo_VillaMontes', 'Purus_Canutama', 'Purus_Labrea',
        'Purus_SeringalDaCaridade', 'Purus_ValparaisoMontante', 'Red_Index',
        'Rufiji_Stigler', 'Salinas_SanAugustin', 'SaoFrancisco_BomJesusDaLapa',
        'SaoFrancisco_Morpara', 'SaptKosi_Baltara', 'SaptKosi_Chatara-Kothu',
        'Selenga_Naushki', 'Solimoes_SantoAntonioDoIca', 'Solimoes_SaoPauloDeOlivenca',
        'Solimoes_Tabatinga', 'Taku_NearTulsequa', 'Tanana_NearHardingLake',
        'Tarauaca_Envira', 'Tista_AndersonBr', 'Tista_Kaunia',
        'Tisza_Vylok', 'Trinity_Romayor', 'Ucayali_Atalaya',
        'Ucayali_Pucallpa', 'Ucayali_Requena', 'Ural_Kushum',
        'White_DevallsBluff', 'White_Petersburg', 'Yukon_NearStevensVillage',
        'Zambezi_Sesheke', 'Beni', 'Beni', 'Beni', 'Beni', 'Beni',
        'Beni', 'Beni', 'Beni', 'Beni', 'Beni', 'Bermejo', 'Bermejo',
        'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo',
        'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo',
        'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo', 'Bermejo',
        'Koyukuk_Huslia', 'Yukon_Beaver', 'Aladan_VerkhoyanskiyPerevoz',
        'Amazonas_Jatuarana', 'Amur_Komsomolsk', 'Benue_Umaisha',
        'BolshayaKet_Rodyonovka', 'Brahmaputra_Pasighat', 'Chari_Bousso',
        'Chari_Guelengdeng', 'Chari_Ndjamena', 'Chari_Sahr', 'Fraser_Hope',
        'Gandak_Devghat', 'Helmand_Kajaki', 'Helmand_Malakhan', 'Indus_Attock',
        'Irtysh_Bobrovsky', 'Irtysh_Hanti-Mansisk', 'Irtysh_Pavlodar',
        'Irtysh_Semiyarskoje', 'Jutai_PortoSeguro', 'Kamchatka_Kozyrevsk',
        'Kan_Kansk', 'MadreDeDios_CachuelaEsperanza', 'Magdalena_Calamar',
        'Magdalena_PuertoBerrio', 'Manas_Mathanguri', 'Mbam_Goura',
        'Mekong_Kratie', 'Niger_Tossaye', 'Ob_Barnaul', 'Ob_Kolpashevo',
        'Ob_Mogochin', 'Ob_Phominskoje', 'Orinoco_CiudadBolivar',
        'Orinoco_Musinacio', 'Paraguay_Asuncion', 'Paraguay_PortoMurtinho',
        'Parana_Chapeton', 'Porcupine_NearFortYukon', 'Sangha_Ouesso',
        'Solimoes_Itapeua', 'Solimoes_Manacapuru', 'SonghuaJiang_Haerbin',
        'Vilyuy_KhatyrykKhoma', 'Yangtze_Datong', 'Yellowstone_NearSidney',
        'Yukon_Eagle', 'Zambezi_LukuluMission', 'Zambezi_Matundo-Cais'
    ]
    
    # Remove duplicates while preserving order
    seen = set()
    folder_names = []
    duplicates_removed = []
    
    for name in raw_folder_names:
        if name not in seen:
            seen.add(name)
            folder_names.append(name)
        else:
            duplicates_removed.append(name)
    
    # Report on duplicates
    if duplicates_removed:
        from collections import Counter
        duplicate_counts = Counter(raw_folder_names)
        print("Duplicate rivers found and removed:")
        for name, count in duplicate_counts.items():
            if count > 1:
                print(f"  - {name}: appeared {count} times, keeping 1")
        print()
    
    # Create the base directory if it doesn't exist
    if not os.path.exists(base_directory):
        os.makedirs(base_directory)
        print(f"Created base directory: {base_directory}\n")
    
    # Create each folder
    created_count = 0
    skipped_count = 0
    subfolders_created = 0
    subfolders_skipped = 0
    
    for folder_name in folder_names:
        folder_path = os.path.join(base_directory, folder_name)
        
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            created_count += 1
            print(f"Created: {folder_name}")
        else:
            skipped_count += 1
            print(f"Already exists: {folder_name}")
        
        # Create subfolder if specified
        if subfolder_name:
            subfolder_path = os.path.join(folder_path, subfolder_name)
            if not os.path.exists(subfolder_path):
                os.makedirs(subfolder_path)
                subfolders_created += 1
            else:
                subfolders_skipped += 1
    
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    print(f"Original list: {len(raw_folder_names)} entries")
    print(f"Duplicates removed: {len(duplicates_removed)}")
    print(f"Unique folders: {len(folder_names)}")
    print(f"✓ Folders created: {created_count}")
    print(f"○ Already existed: {skipped_count}")
    
    if subfolder_name:
        print(f"\nSubfolder '{subfolder_name}':")
        print(f"✓ Created: {subfolders_created}")
        print(f"○ Already existed: {subfolders_skipped}")

if __name__ == "__main__":
    # Specify your target directory here
    target_directory = r"E:\Dissertation\Data\RiverMapping\RiverMasks"  # Change this to your desired path
    
    # Optional: specify a subfolder name to create inside each river folder
    # Set to None if you don't want subfolders
    subfolder = "Cleaned"  # Example: "masks" or "data" or None
    
    create_unique_river_folders(target_directory, subfolder_name=subfolder)

This moves the Greenberg et al 2024 masks from his folder structure to the new folder structure

In [ ]:
import os
import shutil

def move_greenberg_masks():
    """
    Move .tif mask files from Greenberg et al. directory structure to RiverMapping directory.
    Searches through all planform types and river names, finds mask folders, and moves
    all .tif files to the new directory structure.
    """
    
    # Define paths
    source_base = r"E:\Dissertation\Data\Greenberg_etal_2024\RiverData"
    target_base = r"E:\Dissertation\Data\RiverMapping\RiverMasks"
    
    # Define planform types
    planform_types = ["B", "HSW", "LSW", "Me"]
    
    # Counters for summary
    rivers_processed = 0
    files_moved = 0
    rivers_not_found = 0
    rivers_with_multiple_folders = 0
    errors = []
    rivers_with_masks = []
    rivers_without_masks = []
    rivers_with_ambiguous_structure = []
    
    # Get list of all potential river names by scanning all planform directories
    all_river_names = set()
    for planform in planform_types:
        planform_dir = os.path.join(source_base, planform)
        if os.path.exists(planform_dir):
            river_dirs = [d for d in os.listdir(planform_dir) 
                         if os.path.isdir(os.path.join(planform_dir, d))]
            all_river_names.update(river_dirs)
    
    all_river_names = sorted(all_river_names)
    print(f"Found {len(all_river_names)} unique river names across all planform types")
    print(f"Searching through {len(planform_types)} planform types\n")
    
    # Process each river
    for river_name in all_river_names:
        found_masks = False
        
        # Search through planform types for this river
        for planform in planform_types:
            # Construct source path to river directory
            river_dir = os.path.join(source_base, planform, river_name)
            
            if not os.path.exists(river_dir):
                continue
            
            # Check what's inside the river directory
            items_in_river_dir = [d for d in os.listdir(river_dir) 
                                 if os.path.isdir(os.path.join(river_dir, d))]
            
            # Look for potential search locations
            waterlevel_folders = [f for f in items_in_river_dir if f.startswith('WaterLevel')]
            has_mask_direct = 'mask' in items_in_river_dir
            has_river_subfolder = river_name in items_in_river_dir
            
            # Determine the correct path to mask folder
            mask_dir = None
            all_waterlevel_folders = []
            
            # Collect all WaterLevel folders from both levels
            if waterlevel_folders:
                all_waterlevel_folders.extend(waterlevel_folders)
            
            # Also check inside river_name subfolder if it exists
            if has_river_subfolder:
                river_subfolder = os.path.join(river_dir, river_name)
                subfolders = [d for d in os.listdir(river_subfolder) 
                             if os.path.isdir(os.path.join(river_subfolder, d))]
                subfolder_waterlevels = [f for f in subfolders if f.startswith('WaterLevel')]
                all_waterlevel_folders.extend(subfolder_waterlevels)
            
            # Case 1: Multiple WaterLevel folders across both levels - ambiguous
            if len(all_waterlevel_folders) > 1:
                found_masks = True  # Mark as found to avoid "not found" message
                rivers_with_ambiguous_structure.append(river_name)
                rivers_with_multiple_folders += 1
                print(f"⚠ Warning: {river_name} ({planform}) has multiple WaterLevel folders: {all_waterlevel_folders}")
                print(f"   Skipping due to ambiguous structure...")
                break
            
            # Case 2: WaterLevel folder directly in river_dir
            elif len(waterlevel_folders) == 1:
                potential_mask_dir = os.path.join(river_dir, waterlevel_folders[0], "mask")
                if os.path.exists(potential_mask_dir) and os.path.isdir(potential_mask_dir):
                    mask_dir = potential_mask_dir
            
            # Case 3: WaterLevel folder inside river_name subfolder
            elif has_river_subfolder:
                river_subfolder = os.path.join(river_dir, river_name)
                subfolders = [d for d in os.listdir(river_subfolder) 
                             if os.path.isdir(os.path.join(river_subfolder, d))]
                subfolder_waterlevels = [f for f in subfolders if f.startswith('WaterLevel')]
                
                if len(subfolder_waterlevels) == 1:
                    potential_mask_dir = os.path.join(river_subfolder, subfolder_waterlevels[0], "mask")
                    if os.path.exists(potential_mask_dir) and os.path.isdir(potential_mask_dir):
                        mask_dir = potential_mask_dir
                elif 'mask' in subfolders:
                    # No WaterLevel, but mask exists directly in river_name subfolder
                    mask_dir = os.path.join(river_subfolder, "mask")
            
            # Case 4: mask folder directly in river_dir (no river_name subfolder, no WaterLevel)
            elif has_mask_direct:
                mask_dir = os.path.join(river_dir, "mask")
            
            # If we found a mask directory, process it
            if mask_dir:
                # Get all .tif files in the mask directory
                tif_files = [f for f in os.listdir(mask_dir) if f.lower().endswith('.tif')]
                
                if tif_files:
                    found_masks = True
                    
                    try:
                        # Create target directory
                        target_dir = os.path.join(target_base, river_name, "Cleaned")
                        os.makedirs(target_dir, exist_ok=True)
                        
                        # Move each .tif file
                        files_moved_for_river = 0
                        for tif_file in tif_files:
                            source_file = os.path.join(mask_dir, tif_file)
                            target_file = os.path.join(target_dir, tif_file)
                            
                            shutil.move(source_file, target_file)
                            files_moved_for_river += 1
                        
                        files_moved += files_moved_for_river
                        rivers_processed += 1
                        rivers_with_masks.append(river_name)
                        print(f"✓ Moved {files_moved_for_river} file(s): {river_name} (from {planform})")
                        
                    except Exception as e:
                        error_msg = f"{river_name} ({planform})"
                        errors.append(error_msg)
                        print(f"✗ Error: {river_name} ({planform}) - {str(e)}")
                        print(f"   Skipping and continuing with remaining rivers...")
                    
                    # Break after finding first matching planform with masks
                    break
        
        if not found_masks:
            rivers_without_masks.append(river_name)
            rivers_not_found += 1
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    print(f"✓ Rivers with masks processed: {rivers_processed}")
    print(f"✓ Total .tif files moved: {files_moved}")
    print(f"○ Rivers without mask folders: {rivers_not_found}")
    print(f"⚠ Rivers with ambiguous structure (skipped): {rivers_with_multiple_folders}")
    print(f"✗ Errors encountered: {len(errors)}")
    print(f"Total unique rivers scanned: {len(all_river_names)}")
    
    if rivers_with_ambiguous_structure:
        print(f"\n{'='*60}")
        print(f"RIVERS WITH AMBIGUOUS STRUCTURE (Multiple WaterLevel Folders)")
        print(f"{'='*60}")
        for river in rivers_with_ambiguous_structure:
            print(f"  - {river}")
    
    if errors:
        print(f"\n{'='*60}")
        print(f"FAILED RIVERS")
        print(f"{'='*60}")
        for error in errors:
            print(f"  - {error}")
    
    # Optional: Print rivers without masks (useful for debugging)
    if rivers_without_masks and len(rivers_without_masks) <= 20:
        print(f"\n{'='*60}")
        print(f"RIVERS WITHOUT MASK FOLDERS (showing up to 20)")
        print(f"{'='*60}")
        for river in rivers_without_masks[:20]:
            print(f"  - {river}")
        if len(rivers_without_masks) > 20:
            print(f"  ... and {len(rivers_without_masks) - 20} more")

if __name__ == "__main__":
    print("Starting mask file migration...\n")
    move_greenberg_masks()
    print("\nMask migration complete!")

This moves the Zhao et al 2025 masks from his folder structure to the new folder structure

In [ ]:
def move_zhao_masks():
    """
    Move .tif mask files from Zhao et al. directory structure to RiverMapping directory.
    Searches through all planform types and river names, finds PreparedImagery_annual folders,
    and moves all .tif files to the new directory structure.
    """
    
    # Define paths
    source_base = r"E:\Dissertation\Data\Zhao_etal_2025"
    target_base = r"E:\Dissertation\Data\RiverMapping\RiverMasks"
    
    # Define planform types
    planform_types = ["B", "HSW", "LSW", "Me"]
    
    # Counters for summary
    rivers_processed = 0
    files_moved = 0
    rivers_not_found = 0
    errors = []
    rivers_with_masks = []
    rivers_without_masks = []
    
    # Get list of all potential river names by scanning all planform directories
    all_river_names = set()
    for planform in planform_types:
        planform_dir = os.path.join(source_base, planform)
        if os.path.exists(planform_dir):
            river_dirs = [d for d in os.listdir(planform_dir) 
                         if os.path.isdir(os.path.join(planform_dir, d))]
            all_river_names.update(river_dirs)
    
    all_river_names = sorted(all_river_names)
    print(f"Found {len(all_river_names)} unique river names across all planform types")
    print(f"Searching through {len(planform_types)} planform types\n")
    
    # Process each river
    for river_name in all_river_names:
        found_masks = False
        
        # Search through planform types for this river
        for planform in planform_types:
            # Construct source path: planform/river_name/PreparedImagery_annual/river_name
            river_dir = os.path.join(source_base, planform, river_name)
            imagery_dir = os.path.join(river_dir, "PreparedImagery_annual")
            mask_dir = os.path.join(imagery_dir, river_name)
            
            # Check if mask directory exists
            if os.path.exists(mask_dir) and os.path.isdir(mask_dir):
                # Get all .tif files in the directory
                tif_files = [f for f in os.listdir(mask_dir) if f.lower().endswith('.tif')]
                
                if tif_files:
                    found_masks = True
                    
                    try:
                        # Create target directory
                        target_dir = os.path.join(target_base, river_name, "Cleaned")
                        os.makedirs(target_dir, exist_ok=True)
                        
                        # Move each .tif file
                        files_moved_for_river = 0
                        for tif_file in tif_files:
                            source_file = os.path.join(mask_dir, tif_file)
                            target_file = os.path.join(target_dir, tif_file)
                            
                            shutil.move(source_file, target_file)
                            files_moved_for_river += 1
                        
                        files_moved += files_moved_for_river
                        rivers_processed += 1
                        rivers_with_masks.append(river_name)
                        print(f"✓ Moved {files_moved_for_river} file(s): {river_name} (from {planform})")
                        
                    except Exception as e:
                        error_msg = f"{river_name} ({planform})"
                        errors.append(error_msg)
                        print(f"✗ Error: {river_name} ({planform}) - {str(e)}")
                        print(f"   Skipping and continuing with remaining rivers...")
                    
                    # Break after finding first matching planform with masks
                    break
        
        if not found_masks:
            rivers_without_masks.append(river_name)
            rivers_not_found += 1
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    print(f"✓ Rivers with masks processed: {rivers_processed}")
    print(f"✓ Total .tif files moved: {files_moved}")
    print(f"○ Rivers without mask folders: {rivers_not_found}")
    print(f"✗ Errors encountered: {len(errors)}")
    print(f"Total unique rivers scanned: {len(all_river_names)}")
    
    if errors:
        print(f"\n{'='*60}")
        print(f"FAILED RIVERS")
        print(f"{'='*60}")
        for error in errors:
            print(f"  - {error}")
    
    # Optional: Print rivers without masks (useful for debugging)
    if rivers_without_masks and len(rivers_without_masks) <= 20:
        print(f"\n{'='*60}")
        print(f"RIVERS WITHOUT MASK FOLDERS (showing up to 20)")
        print(f"{'='*60}")
        for river in rivers_without_masks[:20]:
            print(f"  - {river}")
        if len(rivers_without_masks) > 20:
            print(f"  ... and {len(rivers_without_masks) - 20} more")


if __name__ == "__main__":
    print("Starting Greenberg mask file migration...\n")
    move_greenberg_masks()
    print("\nGreenberg mask migration complete!")
    
    print("\n" + "="*60)
    print("\nStarting Zhao mask file migration...\n")
    move_zhao_masks()
    print("\nZhao mask migration complete!")

Preliminary QC of river masks

In [ ]:
import os
import numpy as np
import pandas as pd
import rasterio
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from datetime import datetime

def analyze_mask_statistics(mask_dir):
    """
    Analyze all .tif masks in a directory and compute statistics.
    
    Parameters:
    -----------
    mask_dir : str
        Path to directory containing .tif mask files
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with statistics for each mask
    """
    tif_files = sorted([f for f in os.listdir(mask_dir) if f.lower().endswith('.tif')])
    
    if not tif_files:
        return None
    
    stats_list = []
    
    for tif_file in tif_files:
        file_path = os.path.join(mask_dir, tif_file)
        
        try:
            # Suppress the NotGeoreferencedWarning
            import warnings
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=rasterio.errors.NotGeoreferencedWarning)
                with rasterio.open(file_path) as src:
                    data = src.read(1)  # Read first band
            
            # Get unique values to check what's actually in the data
            unique_vals = np.unique(data)
            
            # Convert data to float for consistent handling
            data = data.astype(np.float64)
            
            # Count pixels
            total_pixels = data.size
            
            # Detect if this is a 0/1 mask or 0/255 mask
            # Check if any values are close to 255
            has_255 = np.any(np.abs(data - 255) < 0.01)
            
            if has_255:
                # This is a 0/255 mask (Zhao study)
                is_zero = np.abs(data - 0) < 0.01
                is_wet = np.abs(data - 255) < 0.01
                wet_pixels = np.sum(is_wet)
                dry_pixels = np.sum(is_zero)
                other_pixels = total_pixels - wet_pixels - dry_pixels
            else:
                # This is a 0/1 mask (Greenberg study)
                is_zero = np.abs(data - 0) < 0.01
                is_one = np.abs(data - 1) < 0.01
                wet_pixels = np.sum(is_one)
                dry_pixels = np.sum(is_zero)
                other_pixels = total_pixels - wet_pixels - dry_pixels
            
            # Calculate percentages
            pct_wet = (wet_pixels / total_pixels) * 100 if total_pixels > 0 else 0
            pct_dry = (dry_pixels / total_pixels) * 100 if total_pixels > 0 else 0
            
            stats_list.append({
                'filename': tif_file,
                'total_pixels': total_pixels,
                'wet_pixels': wet_pixels,
                'dry_pixels': dry_pixels,
                'other_pixels': other_pixels,
                'pct_wet': pct_wet,
                'pct_dry': pct_dry,
                'unique_values': str(unique_vals.tolist())
            })
            
        except Exception as e:
            print(f"  Error reading {tif_file}: {e}")
            stats_list.append({
                'filename': tif_file,
                'total_pixels': 0,
                'wet_pixels': 0,
                'dry_pixels': 0,
                'other_pixels': 0,
                'pct_wet': 0,
                'pct_dry': 0,
                'unique_values': '',
                'error': str(e)
            })
    
    return pd.DataFrame(stats_list)

def flag_problematic_masks(df, z_threshold=3):
    """
    Flag masks that deviate significantly from the mean or have other issues.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with mask statistics
    z_threshold : float
        Z-score threshold for flagging outliers
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with added flag columns
    """
    # Calculate z-scores for wet pixels
    mean_wet = df['wet_pixels'].mean()
    std_wet = df['wet_pixels'].std()
    
    if std_wet > 0:
        df['z_score'] = (df['wet_pixels'] - mean_wet) / std_wet
    else:
        df['z_score'] = 0
    
    # Flag different types of problems
    df['flag_all_dry'] = df['wet_pixels'] == 0
    df['flag_all_wet'] = df['pct_wet'] > 99.9
    df['flag_extreme_outlier'] = np.abs(df['z_score']) > z_threshold
    df['flag_non_binary'] = df['other_pixels'] > 0
    df['flag_tiny_wet'] = (df['pct_wet'] < 0.1) & (df['pct_wet'] > 0)
    
    # Overall flag
    df['flagged'] = (df['flag_all_dry'] | df['flag_all_wet'] | 
                     df['flag_extreme_outlier'] | df['flag_non_binary'] | 
                     df['flag_tiny_wet'])
    
    return df

def create_contact_sheet(mask_dir, df, output_path, max_images=40):
    """
    Create a contact sheet showing all masks with flagged ones highlighted.
    
    Parameters:
    -----------
    mask_dir : str
        Path to directory containing masks
    df : pandas.DataFrame
        DataFrame with mask statistics and flags
    output_path : str
        Path to save the contact sheet image
    max_images : int
        Maximum number of images to show
    """
    tif_files = df['filename'].tolist()[:max_images]
    n_images = len(tif_files)
    
    # Calculate grid dimensions
    n_cols = min(8, n_images)
    n_rows = int(np.ceil(n_images / n_cols))
    
    fig = plt.figure(figsize=(20, n_rows * 2.5))
    
    for idx, tif_file in enumerate(tif_files):
        file_path = os.path.join(mask_dir, tif_file)
        
        try:
            # Suppress the NotGeoreferencedWarning
            import warnings
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=rasterio.errors.NotGeoreferencedWarning)
                with rasterio.open(file_path) as src:
                    data = src.read(1)
            
            ax = plt.subplot(n_rows, n_cols, idx + 1)
            ax.imshow(data, cmap='Blues', vmin=0, vmax=1)
            
            # Highlight flagged images with red border
            row = df[df['filename'] == tif_file].iloc[0]
            if row['flagged']:
                for spine in ax.spines.values():
                    spine.set_edgecolor('red')
                    spine.set_linewidth(3)
            
            # Add title with basic info
            title = f"{tif_file[:15]}...\n"
            title += f"Wet: {row['pct_wet']:.1f}%"
            if row['flagged']:
                title += "\n⚠ FLAGGED"
            
            ax.set_title(title, fontsize=8, color='red' if row['flagged'] else 'black')
            ax.axis('off')
            
        except Exception as e:
            print(f"  Error visualizing {tif_file}: {e}")
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

def create_time_series_plot(df, output_path, river_name):
    """
    Create a time series plot of wet pixels over time.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with mask statistics
    output_path : str
        Path to save the plot
    river_name : str
        Name of the river for the title
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    x = np.arange(len(df))
    
    # Plot all points
    ax.plot(x, df['wet_pixels'], 'o-', color='steelblue', markersize=4, label='Normal')
    
    # Highlight flagged points
    flagged_df = df[df['flagged']]
    if not flagged_df.empty:
        flagged_indices = flagged_df.index
        ax.plot(flagged_indices, flagged_df['wet_pixels'], 'ro', 
                markersize=8, label='Flagged', zorder=5)
    
    # Add mean line
    mean_wet = df['wet_pixels'].mean()
    ax.axhline(mean_wet, color='green', linestyle='--', alpha=0.7, label=f'Mean: {mean_wet:.0f}')
    
    # Add ±3 std lines
    std_wet = df['wet_pixels'].std()
    ax.axhline(mean_wet + 3*std_wet, color='red', linestyle=':', alpha=0.5, label='±3 σ')
    ax.axhline(mean_wet - 3*std_wet, color='red', linestyle=':', alpha=0.5)
    
    ax.set_xlabel('Mask Index', fontsize=12)
    ax.set_ylabel('Wet Pixels', fontsize=12)
    ax.set_title(f'{river_name} - Wet Pixel Count Over Time', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

def screen_all_rivers(base_dir, output_dir, z_threshold=3, create_visualizations=True):
    """
    Screen all river mask directories and generate reports.
    
    Parameters:
    -----------
    base_dir : str
        Base directory containing river folders
    output_dir : str
        Directory to save QC reports
    z_threshold : float
        Z-score threshold for flagging outliers
    create_visualizations : bool
        Whether to create contact sheets and plots (slower)
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Get all river directories
    river_dirs = [d for d in os.listdir(base_dir) 
                  if os.path.isdir(os.path.join(base_dir, d))]
    river_dirs = sorted(river_dirs)
    
    print(f"Found {len(river_dirs)} river directories\n")
    
    # Track overall QC status
    qc_summary = []
    
    for river_idx, river_name in enumerate(river_dirs, 1):
        print(f"[{river_idx}/{len(river_dirs)}] Processing {river_name}...", end='', flush=True)
        
        # Path to masks
        mask_dir = os.path.join(base_dir, river_name, "Cleaned")
        
        if not os.path.exists(mask_dir):
            print(f" ⚠ No 'Cleaned' folder found, skipping")
            qc_summary.append({
                'river_name': river_name,
                'status': 'No masks found',
                'total_masks': 0,
                'flagged_masks': 0,
                'qc_complete': False
            })
            continue
        
        # Analyze masks
        print(f" Analyzing...", end='', flush=True)
        df = analyze_mask_statistics(mask_dir)
        
        if df is None or len(df) == 0:
            print(f" ⚠ No .tif files found, skipping")
            qc_summary.append({
                'river_name': river_name,
                'status': 'No .tif files',
                'total_masks': 0,
                'flagged_masks': 0,
                'qc_complete': False
            })
            continue
        
        # Flag problematic masks
        print(f" Flagging...", end='', flush=True)
        df = flag_problematic_masks(df, z_threshold)
        
        # Create river-specific output directory
        river_output_dir = os.path.join(output_dir, river_name)
        os.makedirs(river_output_dir, exist_ok=True)
        
        # Save statistics CSV
        print(f" Saving CSV...", end='', flush=True)
        csv_path = os.path.join(river_output_dir, f"{river_name}_statistics.csv")
        df.to_csv(csv_path, index=False)
        
        if create_visualizations:
            # Create contact sheet
            print(f" Contact sheet...", end='', flush=True)
            contact_sheet_path = os.path.join(river_output_dir, f"{river_name}_contact_sheet.png")
            try:
                create_contact_sheet(mask_dir, df, contact_sheet_path)
            except Exception as e:
                print(f" [Contact sheet error: {e}]", end='', flush=True)
            
            # Create time series plot
            print(f" Time series...", end='', flush=True)
            timeseries_path = os.path.join(river_output_dir, f"{river_name}_timeseries.png")
            try:
                create_time_series_plot(df, timeseries_path, river_name)
            except Exception as e:
                print(f" [Time series error: {e}]", end='', flush=True)
        
        # Summary stats
        total_masks = len(df)
        flagged_count = df['flagged'].sum()
        
        print(f" ✓ {total_masks} masks, {flagged_count} flagged")
        
        qc_summary.append({
            'river_name': river_name,
            'status': 'Processed',
            'total_masks': total_masks,
            'flagged_masks': flagged_count,
            'pct_flagged': (flagged_count / total_masks * 100) if total_masks > 0 else 0,
            'qc_complete': False,
            'date_screened': datetime.now().strftime('%Y-%m-%d'),
            'notes': ''
        })
    
    # Save overall QC tracking file
    qc_df = pd.DataFrame(qc_summary)
    qc_tracking_path = os.path.join(output_dir, 'QC_tracking.csv')
    qc_df.to_csv(qc_tracking_path, index=False)
    
    print(f"\n{'='*60}")
    print(f"SCREENING COMPLETE")
    print(f"{'='*60}")
    print(f"Total rivers screened: {len(river_dirs)}")
    print(f"QC tracking file saved to: {qc_tracking_path}")
    print(f"Individual river reports saved to: {output_dir}")
    print(f"\nRivers with flagged masks:")
    for _, row in qc_df[qc_df['flagged_masks'] > 0].iterrows():
        print(f"  - {row['river_name']}: {row['flagged_masks']}/{row['total_masks']} masks flagged")

if __name__ == "__main__":
    # Set your paths here
    base_directory = r"E:\Dissertation\Data\RiverMapping\RiverMasks"
    output_directory = r"E:\Dissertation\Data\RiverMapping\MaskQC"
    
    # Run the screening
    # Set create_visualizations=False for faster CSV-only mode
    screen_all_rivers(base_directory, output_directory, z_threshold=3, create_visualizations=False)

## Interactive QC viewer

In [1]:
%matplotlib qt
# or
%matplotlib tk

import os
import pandas as pd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
from datetime import datetime
import warnings

class JupyterMaskQCViewer:
    """
    Jupyter notebook widget-based viewer for QC'ing river masks.
    Similar interface to the GEE validation script.
    """
    
    def __init__(self, qc_dir, base_mask_dir):
        self.qc_dir = qc_dir
        self.base_mask_dir = base_mask_dir
        self.current_river_idx = 0
        self.current_mask_idx = 0
        
        # Load QC tracking
        self.qc_tracking = pd.read_csv(os.path.join(qc_dir, 'QC_tracking.csv'))
        
        # Filter to rivers that need QC (prioritize flagged, then incomplete)
        self.rivers_to_qc = self.qc_tracking[
            (self.qc_tracking['flagged_masks'] > 0) | 
            (self.qc_tracking['qc_complete'] == False)
        ].copy()
        
        self.rivers_to_qc = self.rivers_to_qc.sort_values('pct_flagged', ascending=False)
        
        # Initialize results tracking
        self.qc_results_path = os.path.join(qc_dir, 'mask_qc_results.csv')
        try:
            self.qc_results = pd.read_csv(self.qc_results_path)
            print(f"Loaded existing QC results: {len(self.qc_results)} classifications")
        except FileNotFoundError:
            self.qc_results = pd.DataFrame(columns=[
                'River_Name', 'Filename', 'Classification', 
                'Wet_Pct', 'Z_Score', 'Flagged', 'Timestamp'
            ])
            print("Created new QC results DataFrame")
        
        print(f"Found {len(self.rivers_to_qc)} rivers needing QC")
        print(f"Rivers with flagged masks: {(self.rivers_to_qc['flagged_masks'] > 0).sum()}\n")
        
        self.setup_widgets()
    
    def load_river_data(self, river_name):
        """Load statistics CSV for a river."""
        csv_path = os.path.join(self.qc_dir, river_name, f"{river_name}_statistics.csv")
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            # Sort by flagged first, then by z_score magnitude
            df['abs_z'] = df['z_score'].abs()
            df = df.sort_values(['flagged', 'abs_z'], ascending=[False, False])
            return df
        return None
    
    def setup_widgets(self):
        """Create all UI widgets."""
        # Info display
        self.info_text = widgets.HTML()
        
        # Mask display output
        self.mask_output = widgets.Output()
        
        # Classification buttons
        self.btn_good = widgets.Button(
            description='✓ Good Mask',
            button_style='success',
            layout=widgets.Layout(width='150px')
        )
        self.btn_delete = widgets.Button(
            description='✗ Delete',
            button_style='danger',
            layout=widgets.Layout(width='150px')
        )
        self.btn_review = widgets.Button(
            description='? Further Review',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        )
        
        # Navigation buttons
        self.btn_prev = widgets.Button(
            description='← Prev Mask',
            layout=widgets.Layout(width='120px')
        )
        self.btn_next = widgets.Button(
            description='Next Mask →',
            layout=widgets.Layout(width='120px')
        )
        self.btn_next_river = widgets.Button(
            description='Next River ⇒',
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        self.btn_save = widgets.Button(
            description='💾 Save Progress',
            button_style='',
            layout=widgets.Layout(width='120px')
        )
        
        # Connect button callbacks
        self.btn_good.on_click(lambda b: self.classify('Good'))
        self.btn_delete.on_click(lambda b: self.classify('Delete'))
        self.btn_review.on_click(lambda b: self.classify('Review'))
        self.btn_prev.on_click(lambda b: self.navigate(-1))
        self.btn_next.on_click(lambda b: self.navigate(1))
        self.btn_next_river.on_click(lambda b: self.finish_river())
        self.btn_save.on_click(lambda b: self.save_results())
        
        # Layout
        self.ui = widgets.VBox([
            self.info_text,
            widgets.HBox([self.btn_good, self.btn_delete, self.btn_review]),
            widgets.HBox([self.btn_prev, self.btn_next, self.btn_next_river, self.btn_save])
        ])
    
    def show_river(self, river_idx):
        """Load and start showing masks for a specific river."""
        if river_idx >= len(self.rivers_to_qc):
            self.info_text.value = "<h3 style='color: green;'>✓ All rivers QC'd!</h3>"
            self.mask_output.clear_output()
            return
        
        self.current_river_idx = river_idx
        river_row = self.rivers_to_qc.iloc[river_idx]
        self.current_river_name = river_row['river_name']
        
        # Load river data
        self.current_df = self.load_river_data(self.current_river_name)
        if self.current_df is None:
            print(f"No statistics found for {self.current_river_name}, skipping...")
            self.show_river(river_idx + 1)
            return
        
        self.current_mask_idx = 0
        print(f"\n{'='*60}")
        print(f"River {river_idx + 1}/{len(self.rivers_to_qc)}: {self.current_river_name}")
        print(f"Total masks: {river_row['total_masks']}, Flagged: {river_row['flagged_masks']}")
        print(f"{'='*60}\n")
        
        self.show_mask()
    
    def show_mask(self):
        """Display the current mask."""
        if self.current_mask_idx >= len(self.current_df):
            self.finish_river()
            return
        
        row = self.current_df.iloc[self.current_mask_idx]
        mask_path = os.path.join(self.base_mask_dir, self.current_river_name, 
                                 "Cleaned", row['filename'])
        
        # Update info text
        river_row = self.rivers_to_qc.iloc[self.current_river_idx]
        flag_text = ""
        if row['flagged']:
            flags = []
            if row['flag_all_dry']: flags.append("All Dry")
            if row['flag_all_wet']: flags.append("All Wet")
            if row['flag_extreme_outlier']: flags.append("Extreme Outlier")
            if row['flag_non_binary']: flags.append("Non-Binary")
            if row['flag_tiny_wet']: flags.append("Tiny Wet Area")
            flag_text = f"<br><span style='color: red; font-weight: bold;'>⚠ FLAGGED: {', '.join(flags)}</span>"
        
        self.info_text.value = f"""
        <h3>{self.current_river_name} - Mask {self.current_mask_idx + 1}/{len(self.current_df)}</h3>
        <b>River:</b> {self.current_river_idx + 1}/{len(self.rivers_to_qc)} | 
        <b>File:</b> {row['filename']}<br>
        <b>Wet:</b> {row['pct_wet']:.2f}% | <b>Z-score:</b> {row['z_score']:.2f}{flag_text}
        """
        
        # Load and display mask in a new popup figure window
        try:
            # Close any existing figures first
            plt.close('all')
            
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=rasterio.errors.NotGeoreferencedWarning)
                with rasterio.open(mask_path) as src:
                    data = src.read(1)
            
            # Normalize for display
            data_normalized = data / data.max() if data.max() > 0 else data
            
            # Create a new figure window (popup)
            plt.figure(figsize=(12, 10))
            ax = plt.gca()
            im = ax.imshow(data_normalized, cmap='Blues', vmin=0, vmax=1)
            
            # Title with all info
            title = f"{self.current_river_name} - Mask {self.current_mask_idx + 1}/{len(self.current_df)}\n"
            title += f"{row['filename']}\n"
            title += f"Wet: {row['pct_wet']:.2f}% | Z-score: {row['z_score']:.2f}"
            if row['flagged']:
                title += " | ⚠ FLAGGED"
            
            ax.set_title(title, fontsize=14, fontweight='bold',
                        color='red' if row['flagged'] else 'black')
            ax.axis('off')
            
            # Add colorbar
            plt.colorbar(im, ax=ax, label='Water (0=Dry, 1=Wet)', fraction=0.046)
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"Error loading mask: {e}")
    
    def classify(self, classification):
        """Record classification and advance to next mask."""
        row = self.current_df.iloc[self.current_mask_idx]
        
        # Record result
        new_row = {
            'River_Name': self.current_river_name,
            'Filename': row['filename'],
            'Classification': classification,
            'Wet_Pct': row['pct_wet'],
            'Z_Score': row['z_score'],
            'Flagged': row['flagged'],
            'Timestamp': datetime.now().isoformat()
        }
        
        self.qc_results = pd.concat([self.qc_results, pd.DataFrame([new_row])], ignore_index=True)
        
        # Auto-save every 10 classifications
        if len(self.qc_results) % 10 == 0:
            self.save_results()
            print(f"Auto-saved: {len(self.qc_results)} classifications")
        
        # Move to next mask
        self.navigate(1)
    
    def navigate(self, direction):
        """Navigate between masks."""
        self.current_mask_idx = max(0, min(len(self.current_df) - 1, 
                                            self.current_mask_idx + direction))
        self.show_mask()
    
    def finish_river(self):
        """Complete current river and move to next."""
        # Mark river as QC complete
        self.qc_tracking.loc[
            self.qc_tracking['river_name'] == self.current_river_name,
            'qc_complete'
        ] = True
        
        # Count classifications for this river
        river_results = self.qc_results[self.qc_results['River_Name'] == self.current_river_name]
        delete_count = (river_results['Classification'] == 'Delete').sum()
        
        self.qc_tracking.loc[
            self.qc_tracking['river_name'] == self.current_river_name,
            'notes'
        ] = f"{len(river_results)} reviewed, {delete_count} to delete"
        
        # Save progress
        self.save_results()
        self.qc_tracking.to_csv(os.path.join(self.qc_dir, 'QC_tracking.csv'), index=False)
        
        print(f"\n✓ Completed {self.current_river_name}")
        print(f"  Reviewed: {len(river_results)}, To Delete: {delete_count}\n")
        
        # Move to next river
        self.show_river(self.current_river_idx + 1)
    
    def save_results(self):
        """Save QC results to CSV."""
        self.qc_results.to_csv(self.qc_results_path, index=False)
        print(f"  Progress saved: {len(self.qc_results)} total classifications")
    
    def start(self):
        """Display the UI and start QC process."""
        display(self.ui)
        self.show_river(0)
        print("\nQC interface ready!")
        print("Classify each mask as: Good, Delete, or Review")
        print("Use Next River button to complete a river and move to the next\n")

# Usage:
viewer = JupyterMaskQCViewer(
    qc_dir=r"E:\Dissertation\Data\RiverMapping\MaskQC",
    base_mask_dir=r"E:\Dissertation\Data\RiverMapping\RiverMasks"  # Changed mask_dir to base_mask_dir
)
viewer.start()

Created new QC results DataFrame
Found 135 rivers needing QC
Rivers with flagged masks: 81




River 1/135: Logone_Bongor
Total masks: 37, Flagged: 21


QC interface ready!
Classify each mask as: Good, Delete, or Review
Use Next River button to complete a river and move to the next



## This actually deletes masks that are both flagged and marked for deletion
The code first runs a "dry run" to show all masks marked for deletion

In [3]:
import os
import pandas as pd
from pathlib import Path

def delete_flagged_masks(results_csv, mask_base_dir, dry_run=True):
    """
    Delete mask files that are both flagged (TRUE) and marked for deletion.
    
    Parameters:
    -----------
    results_csv : str
        Path to the mask_qc_results.csv file
    mask_base_dir : str
        Base directory containing river mask folders
    dry_run : bool
        If True, only shows what would be deleted without actually deleting
    """
    
    # Load QC results
    df = pd.read_csv(results_csv)
    
    print(f"Loaded {len(df)} QC records")
    print(f"{'='*60}\n")
    
    # Filter for masks that are both Flagged=TRUE and Classification=Delete
    to_delete = df[(df['Flagged'] == True) & (df['Classification'] == 'Delete')]
    
    print(f"Found {len(to_delete)} masks marked as Flagged AND Delete")
    print(f"{'='*60}\n")
    
    # Track results
    deleted_count = 0
    not_found_count = 0
    error_count = 0
    
    deleted_files = []
    not_found_files = []
    errors = []
    
    # Process each file
    for idx, row in to_delete.iterrows():
        river_name = row['River_Name']
        filename = row['Filename']
        
        # Construct file path
        file_path = os.path.join(mask_base_dir, river_name, "Cleaned", filename)
        
        if os.path.exists(file_path):
            if dry_run:
                print(f"[DRY RUN] Would delete: {river_name}/{filename}")
                deleted_count += 1
                deleted_files.append(f"{river_name}/{filename}")
            else:
                try:
                    os.remove(file_path)
                    print(f"✓ Deleted: {river_name}/{filename}")
                    deleted_count += 1
                    deleted_files.append(f"{river_name}/{filename}")
                except Exception as e:
                    print(f"✗ Error deleting {river_name}/{filename}: {e}")
                    error_count += 1
                    errors.append(f"{river_name}/{filename}: {e}")
        else:
            print(f"⚠ Not found: {river_name}/{filename}")
            not_found_count += 1
            not_found_files.append(f"{river_name}/{filename}")
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    if dry_run:
        print(f"[DRY RUN MODE - No files were actually deleted]")
    print(f"✓ Files deleted (or would be deleted): {deleted_count}")
    print(f"⚠ Files not found: {not_found_count}")
    print(f"✗ Errors encountered: {error_count}")
    
    # Show breakdown by river
    if deleted_count > 0:
        print(f"\n{'='*60}")
        print(f"DELETIONS BY RIVER")
        print(f"{'='*60}")
        river_counts = to_delete[to_delete['Filename'].isin([f.split('/')[-1] for f in deleted_files])]['River_Name'].value_counts()
        for river, count in river_counts.items():
            print(f"  {river}: {count} masks")
    
    # Show not found files if any
    if not_found_count > 0 and not_found_count <= 20:
        print(f"\n{'='*60}")
        print(f"FILES NOT FOUND")
        print(f"{'='*60}")
        for file in not_found_files[:20]:
            print(f"  {file}")
        if not_found_count > 20:
            print(f"  ... and {not_found_count - 20} more")
    
    # Show errors if any
    if error_count > 0:
        print(f"\n{'='*60}")
        print(f"ERRORS")
        print(f"{'='*60}")
        for error in errors:
            print(f"  {error}")
    
    if dry_run:
        print(f"\n{'='*60}")
        print(f"This was a DRY RUN. No files were actually deleted.")
        print(f"Set dry_run=False to actually delete the files.")
        print(f"{'='*60}")

if __name__ == "__main__":
    # Set your paths
    results_csv_path = r"E:\Dissertation\Data\RiverMapping\MaskQC\mask_qc_results.csv"
    mask_directory = r"E:\Dissertation\Data\RiverMapping\RiverMasks"
    
    # Run in DRY RUN mode first to see what would be deleted
    print("Running in DRY RUN mode...\n")
    delete_flagged_masks(results_csv_path, mask_directory, dry_run=True)
    
    # Uncomment the lines below to actually delete the files after reviewing the dry run
    print("\n\n" + "="*60)
    print("Running ACTUAL deletion...")
    print("="*60 + "\n")
    delete_flagged_masks(results_csv_path, mask_directory, dry_run=False)

Running in DRY RUN mode...

Loaded 3717 QC records

Found 450 masks marked as Flagged AND Delete

[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1985_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1988_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1989_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1990_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1991_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1992_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1993_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1994_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1995_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1996_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone_Bongor_1997_01-01_12-31_mask.tif
[DRY RUN] Would delete: Logone_Bongor/Logone

## This deletes masks that are not flagged, but marked for deletion and have a wetted area z-score of > 2 or < -2

In [5]:
import os
import pandas as pd
from pathlib import Path

def delete_unflagged_outliers(results_csv, mask_base_dir, dry_run=True):
    """
    Delete mask files that are NOT flagged but marked for deletion with |z-score| > 2.
    
    Parameters:
    -----------
    results_csv : str
        Path to the mask_qc_results.csv file
    mask_base_dir : str
        Base directory containing river mask folders
    dry_run : bool
        If True, only shows what would be deleted without actually deleting
    """
    
    # Load QC results
    df = pd.read_csv(results_csv)
    
    print(f"Loaded {len(df)} QC records")
    print(f"{'='*60}\n")
    
    # Filter for masks that are:
    # - Flagged = FALSE (not flagged)
    # - Classification = Delete
    # - Z_Score > 2 OR Z_Score < -2
    to_delete = df[
        (df['Flagged'] == False) & 
        (df['Classification'] == 'Delete') &
        ((df['Z_Score'] > 2) | (df['Z_Score'] < -2))
    ]
    
    print(f"Found {len(to_delete)} masks that are:")
    print(f"  - NOT flagged (Flagged = FALSE)")
    print(f"  - Marked for deletion (Classification = Delete)")
    print(f"  - Extreme outliers (|Z-score| > 2)")
    print(f"{'='*60}\n")
    
    # Show Z-score distribution of files to delete
    if len(to_delete) > 0:
        print("Z-score distribution of masks to delete:")
        print(f"  Min Z-score: {to_delete['Z_Score'].min():.2f}")
        print(f"  Max Z-score: {to_delete['Z_Score'].max():.2f}")
        print(f"  Mean Z-score: {to_delete['Z_Score'].mean():.2f}")
        print(f"  Masks with Z > 2: {(to_delete['Z_Score'] > 2).sum()}")
        print(f"  Masks with Z < -2: {(to_delete['Z_Score'] < -2).sum()}")
        print(f"{'='*60}\n")
    
    # Track results
    deleted_count = 0
    not_found_count = 0
    error_count = 0
    
    deleted_files = []
    not_found_files = []
    errors = []
    
    # Process each file
    for idx, row in to_delete.iterrows():
        river_name = row['River_Name']
        filename = row['Filename']
        z_score = row['Z_Score']
        
        # Construct file path
        file_path = os.path.join(mask_base_dir, river_name, "Cleaned", filename)
        
        if os.path.exists(file_path):
            if dry_run:
                print(f"[DRY RUN] Would delete: {river_name}/{filename} (Z={z_score:.2f})")
                deleted_count += 1
                deleted_files.append(f"{river_name}/{filename}")
            else:
                try:
                    os.remove(file_path)
                    print(f"✓ Deleted: {river_name}/{filename} (Z={z_score:.2f})")
                    deleted_count += 1
                    deleted_files.append(f"{river_name}/{filename}")
                except Exception as e:
                    print(f"✗ Error deleting {river_name}/{filename}: {e}")
                    error_count += 1
                    errors.append(f"{river_name}/{filename}: {e}")
        else:
            print(f"⚠ Not found: {river_name}/{filename}")
            not_found_count += 1
            not_found_files.append(f"{river_name}/{filename}")
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    if dry_run:
        print(f"[DRY RUN MODE - No files were actually deleted]")
    print(f"✓ Files deleted (or would be deleted): {deleted_count}")
    print(f"⚠ Files not found: {not_found_count}")
    print(f"✗ Errors encountered: {error_count}")
    
    # Show breakdown by river
    if deleted_count > 0:
        print(f"\n{'='*60}")
        print(f"DELETIONS BY RIVER")
        print(f"{'='*60}")
        river_counts = to_delete[to_delete['Filename'].isin([f.split('/')[-1] for f in deleted_files])]['River_Name'].value_counts()
        for river, count in river_counts.items():
            print(f"  {river}: {count} masks")
    
    # Show not found files if any
    if not_found_count > 0 and not_found_count <= 20:
        print(f"\n{'='*60}")
        print(f"FILES NOT FOUND")
        print(f"{'='*60}")
        for file in not_found_files[:20]:
            print(f"  {file}")
        if not_found_count > 20:
            print(f"  ... and {not_found_count - 20} more")
    
    # Show errors if any
    if error_count > 0:
        print(f"\n{'='*60}")
        print(f"ERRORS")
        print(f"{'='*60}")
        for error in errors:
            print(f"  {error}")
    
    if dry_run:
        print(f"\n{'='*60}")
        print(f"This was a DRY RUN. No files were actually deleted.")
        print(f"Set dry_run=False to actually delete the files.")
        print(f"{'='*60}")

if __name__ == "__main__":
    # Set your paths
    results_csv_path = r"E:\Dissertation\Data\RiverMapping\MaskQC\mask_qc_results.csv"
    mask_directory = r"E:\Dissertation\Data\RiverMapping\RiverMasks"
    
    # Run in DRY RUN mode first to see what would be deleted
    print("Running in DRY RUN mode...\n")
    delete_unflagged_outliers(results_csv_path, mask_directory, dry_run=True)
    
    # Uncomment the lines below to actually delete the files after reviewing the dry run
    print("\n\n" + "="*60)
    print("Running ACTUAL deletion...")
    print("="*60 + "\n")
    delete_unflagged_outliers(results_csv_path, mask_directory, dry_run=False)

Running in DRY RUN mode...

Loaded 3717 QC records

Found 60 masks that are:
  - NOT flagged (Flagged = FALSE)
  - Marked for deletion (Classification = Delete)
  - Extreme outliers (|Z-score| > 2)

Z-score distribution of masks to delete:
  Min Z-score: -2.87
  Max Z-score: 2.87
  Mean Z-score: 0.44
  Masks with Z > 2: 35
  Masks with Z < -2: 25

[DRY RUN] Would delete: Ob_Kolpashevo/Ob_Kolpashevo_2003_01_01_2003_12_31_mask_thresh.tif (Z=2.33)
[DRY RUN] Would delete: Ob_Barnaul/Ob_Barnaul_1997_01_01_1997_12_31_mask_thresh.tif (Z=2.55)
[DRY RUN] Would delete: Paraguay_PortoMurtinho/Paraguay_PortoMurtinho_1995_01_01_1995_12_31_mask_thresh.tif (Z=2.87)
[DRY RUN] Would delete: Rufiji_Stigler/Rufiji_Stigler_2010_01-01_12-01_mask.tif (Z=2.45)
[DRY RUN] Would delete: Yukon_Eagle/Yukon_Eagle_1985_01_01_1985_12_31_mask_thresh.tif (Z=2.79)
[DRY RUN] Would delete: Yukon_Eagle/Yukon_Eagle_1991_01_01_1991_12_31_mask_thresh.tif (Z=-2.49)
[DRY RUN] Would delete: Yukon_Eagle/Yukon_Eagle_1993_01_01_19

## Prints out the river reaches with less than 20 masks remaining

In [7]:
import os
import pandas as pd

def count_remaining_masks(mask_base_dir, output_csv=None):
    """
    Count remaining .tif mask files for each river and identify rivers with < 20 masks.
    
    Parameters:
    -----------
    mask_base_dir : str
        Base directory containing river mask folders
    output_csv : str, optional
        If provided, save the results to this CSV file
    """
    
    # Get all river directories
    river_dirs = [d for d in os.listdir(mask_base_dir) 
                  if os.path.isdir(os.path.join(mask_base_dir, d))]
    river_dirs = sorted(river_dirs)
    
    print(f"Scanning {len(river_dirs)} river directories...\n")
    print(f"{'='*60}")
    
    # Track counts
    mask_counts = []
    
    for river_name in river_dirs:
        # Path to Cleaned folder
        cleaned_dir = os.path.join(mask_base_dir, river_name, "Cleaned")
        
        if os.path.exists(cleaned_dir):
            # Count .tif files
            tif_files = [f for f in os.listdir(cleaned_dir) 
                        if f.lower().endswith('.tif')]
            count = len(tif_files)
        else:
            count = 0
        
        mask_counts.append({
            'River_Name': river_name,
            'Mask_Count': count
        })
        
        print(f"{river_name}: {count} masks")
    
    # Convert to DataFrame
    df = pd.DataFrame(mask_counts)
    
    # Print summary statistics
    print(f"\n{'='*60}")
    print(f"SUMMARY STATISTICS")
    print(f"{'='*60}")
    print(f"Total rivers: {len(df)}")
    print(f"Total masks: {df['Mask_Count'].sum()}")
    print(f"Mean masks per river: {df['Mask_Count'].mean():.1f}")
    print(f"Median masks per river: {df['Mask_Count'].median():.1f}")
    print(f"Min masks: {df['Mask_Count'].min()}")
    print(f"Max masks: {df['Mask_Count'].max()}")
    
    # Rivers with < 20 masks
    low_count_rivers = df[df['Mask_Count'] < 20].sort_values('Mask_Count')
    
    print(f"\n{'='*60}")
    print(f"RIVERS WITH < 20 MASKS ({len(low_count_rivers)} rivers)")
    print(f"{'='*60}")
    
    if len(low_count_rivers) > 0:
        for idx, row in low_count_rivers.iterrows():
            print(f"{row['River_Name']}: {row['Mask_Count']} masks")
    else:
        print("No rivers with < 20 masks")
    
    # Rivers with 0 masks
    zero_count_rivers = df[df['Mask_Count'] == 0]
    
    if len(zero_count_rivers) > 0:
        print(f"\n{'='*60}")
        print(f"RIVERS WITH 0 MASKS ({len(zero_count_rivers)} rivers)")
        print(f"{'='*60}")
        for idx, row in zero_count_rivers.iterrows():
            print(f"  {row['River_Name']}")
    
    # Distribution by mask count ranges
    print(f"\n{'='*60}")
    print(f"DISTRIBUTION BY MASK COUNT")
    print(f"{'='*60}")
    print(f"0 masks: {(df['Mask_Count'] == 0).sum()} rivers")
    print(f"1-10 masks: {((df['Mask_Count'] >= 1) & (df['Mask_Count'] <= 10)).sum()} rivers")
    print(f"11-20 masks: {((df['Mask_Count'] >= 11) & (df['Mask_Count'] <= 20)).sum()} rivers")
    print(f"21-30 masks: {((df['Mask_Count'] >= 21) & (df['Mask_Count'] <= 30)).sum()} rivers")
    print(f"31-40 masks: {((df['Mask_Count'] >= 31) & (df['Mask_Count'] <= 40)).sum()} rivers")
    print(f"41+ masks: {(df['Mask_Count'] > 40).sum()} rivers")
    
    # Save to CSV if requested
    if output_csv:
        df.to_csv(output_csv, index=False)
        print(f"\n{'='*60}")
        print(f"Results saved to: {output_csv}")
    
    return df

if __name__ == "__main__":
    # Set your paths
    mask_directory = r"E:\Dissertation\Data\RiverMapping\RiverMasks"
    output_file = r"E:\Dissertation\Data\RiverMapping\MaskQC\remaining_masks_count.csv"
    
    # Run the count
    df = count_remaining_masks(mask_directory, output_csv=output_file)

Scanning 135 river directories...

Aladan_VerkhoyanskiyPerevoz: 18 masks
Amazonas_Jatuarana: 36 masks
Amazonas_Tamshiyacu: 30 masks
AmuDarya_Kerki: 28 masks
Amur_Khabarovsk: 26 masks
Amur_Komsomolsk: 37 masks
Amyl_Kachulka: 23 masks
Apalachicola_NearBlountstown: 36 masks
Araguaia_Aruana: 34 masks
Araguaia_LuizAlves: 35 masks
Araguaia_SaoFelixDoAraguaia: 36 masks
Beas_MandiPlain: 28 masks
Beni: 0 masks
Beni_Rurrenabaque: 35 masks
Benue_Ibi: 12 masks
Benue_Umaisha: 19 masks
Bermejo: 0 masks
Bhareli_NTRoadCrossing: 28 masks
BolshayaKet_Rodyonovka: 22 masks
Brahmaputra_Bahadurabad: 33 masks
Brahmaputra_Pandu: 31 masks
Brahmaputra_Pasighat: 37 masks
Brahmaputra_Yangcun: 32 masks
Chari_Bousso: 20 masks
Chari_Guelengdeng: 18 masks
Chari_Ndjamena: 18 masks
Chari_Sahr: 20 masks
Chenab_Akhnoor: 19 masks
Cuiaba_PortoDoAlegre: 34 masks
Demini_PostoAjuricaba: 34 masks
Fraser_Hope: 38 masks
Gandak_Devghat: 33 masks
Gandak_Dumariaghat: 31 masks
Gandak_Triveni: 24 masks
Ganges_Paksey: 34 masks
Guapore

## This returns unique pixel values for all masks (should be 1s and 0s only)

In [9]:
import os
import numpy as np
import rasterio
import warnings
from collections import defaultdict

def check_mask_pixel_values(mask_base_dir):
    """
    Check one mask file per river for pixel values. Should only contain 0 and 1 (or 0 and 255).
    Flag any masks with unexpected values.
    
    Parameters:
    -----------
    mask_base_dir : str
        Base directory containing river mask folders
        
    Returns:
    --------
    dict
        Dictionary with all unique values found across sampled masks
    """
    
    # Get all river directories
    river_dirs = [d for d in os.listdir(mask_base_dir) 
                  if os.path.isdir(os.path.join(mask_base_dir, d))]
    river_dirs = sorted(river_dirs)
    
    print(f"Checking one mask from each of {len(river_dirs)} rivers...\n")
    print(f"{'='*60}\n")
    
    # Track unique values across all masks
    all_unique_values = set()
    
    # Track problematic masks
    problematic_masks = []
    
    # Track value encoding types
    encoding_types = defaultdict(int)  # 0/1 vs 0/255
    
    # Counters
    rivers_checked = 0
    rivers_skipped = 0
    errors = 0
    
    for river_name in river_dirs:
        # Path to Cleaned folder
        cleaned_dir = os.path.join(mask_base_dir, river_name, "Cleaned")
        
        if not os.path.exists(cleaned_dir):
            rivers_skipped += 1
            continue
        
        # Get all .tif files
        tif_files = [f for f in os.listdir(cleaned_dir) 
                    if f.lower().endswith('.tif')]
        
        if len(tif_files) == 0:
            rivers_skipped += 1
            continue
        
        # Check only the first mask file
        tif_file = tif_files[0]
        file_path = os.path.join(cleaned_dir, tif_file)
        
        try:
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=rasterio.errors.NotGeoreferencedWarning)
                with rasterio.open(file_path) as src:
                    data = src.read(1)
            
            # Get unique values
            unique_vals = np.unique(data)
            
            # Add to overall set
            for val in unique_vals:
                all_unique_values.add(val)
            
            # Check if values are expected
            unique_set = set(unique_vals)
            
            # Determine encoding type
            if unique_set <= {0, 1}:
                encoding_types['0/1'] += 1
                print(f"✓ {river_name}: 0/1 encoding")
            elif unique_set <= {0, 255}:
                encoding_types['0/255'] += 1
                print(f"✓ {river_name}: 0/255 encoding")
            else:
                # Unexpected values found
                problematic_masks.append({
                    'river': river_name,
                    'filename': tif_file,
                    'unique_values': sorted(unique_vals.tolist())
                })
                print(f"⚠ FLAGGED: {river_name}/{tif_file}")
                print(f"   Unexpected values: {sorted(unique_vals.tolist())}\n")
            
            rivers_checked += 1
            
        except Exception as e:
            errors += 1
            print(f"✗ Error reading {river_name}/{tif_file}: {e}")
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    print(f"Total rivers: {len(river_dirs)}")
    print(f"Rivers checked: {rivers_checked}")
    print(f"Rivers skipped (no masks): {rivers_skipped}")
    print(f"Errors encountered: {errors}")
    print(f"\nUnique pixel values found across sampled masks:")
    print(f"  {sorted(all_unique_values)}")
    
    print(f"\nEncoding types:")
    print(f"  0/1 encoding: {encoding_types['0/1']} rivers")
    print(f"  0/255 encoding: {encoding_types['0/255']} rivers")
    
    print(f"\n{'='*60}")
    print(f"PROBLEMATIC MASKS")
    print(f"{'='*60}")
    print(f"Rivers with unexpected values: {len(problematic_masks)}")
    
    if len(problematic_masks) == 0:
        print("✓ All sampled masks contain only expected values (0/1 or 0/255)")
    else:
        print(f"\n{len(problematic_masks)} rivers have unexpected pixel values:")
        for mask in problematic_masks:
            print(f"\n  {mask['river']}/{mask['filename']}")
            print(f"    Values: {mask['unique_values']}")
    
    # Return results
    return {
        'all_unique_values': sorted(all_unique_values),
        'problematic_masks': problematic_masks,
        'encoding_types': dict(encoding_types),
        'rivers_checked': rivers_checked,
        'rivers_skipped': rivers_skipped,
        'errors': errors
    }

if __name__ == "__main__":
    # Set your path
    mask_directory = r"E:\Dissertation\Data\RiverMapping\RiverMasks"
    
    # Run the check
    results = check_mask_pixel_values(mask_directory)
    
    # Print final verdict
    print(f"\n{'='*60}")
    print(f"FINAL VERDICT")
    print(f"{'='*60}")
    if len(results['problematic_masks']) == 0:
        print("✓ ALL MASKS PASSED: Only binary values (0/1 or 0/255) found")
    else:
        print(f"⚠ {len(results['problematic_masks'])} MASKS FLAGGED with unexpected values")
        print("Review the problematic masks listed above")

Checking one mask from each of 135 rivers...


✓ Aladan_VerkhoyanskiyPerevoz: 0/255 encoding
✓ Amazonas_Jatuarana: 0/255 encoding
✓ Amazonas_Tamshiyacu: 0/1 encoding
✓ AmuDarya_Kerki: 0/1 encoding
✓ Amur_Khabarovsk: 0/1 encoding
✓ Amur_Komsomolsk: 0/255 encoding
✓ Amyl_Kachulka: 0/1 encoding
✓ Apalachicola_NearBlountstown: 0/1 encoding
✓ Araguaia_Aruana: 0/1 encoding
✓ Araguaia_LuizAlves: 0/1 encoding
✓ Araguaia_SaoFelixDoAraguaia: 0/1 encoding
✓ Beas_MandiPlain: 0/1 encoding
✓ Beni_Rurrenabaque: 0/1 encoding
✓ Benue_Ibi: 0/1 encoding
✓ Benue_Umaisha: 0/255 encoding
✓ Bhareli_NTRoadCrossing: 0/1 encoding
✓ BolshayaKet_Rodyonovka: 0/255 encoding
✓ Brahmaputra_Bahadurabad: 0/1 encoding
✓ Brahmaputra_Pandu: 0/1 encoding
✓ Brahmaputra_Pasighat: 0/255 encoding
✓ Brahmaputra_Yangcun: 0/1 encoding
✓ Chari_Bousso: 0/255 encoding
✓ Chari_Guelengdeng: 0/255 encoding
✓ Chari_Ndjamena: 0/255 encoding
✓ Chari_Sahr: 0/255 encoding
✓ Chenab_Akhnoor: 0/1 encoding
✓ Cuiaba_PortoDoAlegre: 0/1 encoding
✓